# RFMIP-CLEAR-SKY example (test02 without NN)

*Last edited: 2024-11-12*

Compute clear-sky longwave fluxes. Based on git branch `nn_devel` from https://github.com/peterukk/

Instructions for building are in: `/home/x/git/radnn/ukk22test02/examples/rfmip-clear-sky/Readme.md`

## Notes

- The `rrttmgp_rfmip_{lw|sw}.F90` file has been changed to include `use_rrtmgp_nn = .false.`

- From the Readme.md: "The example programs have been modified to allow neural networks to be used by setting `use_rrtmgp_nn = .true.` (was `use_nn = .true.`) in `rrttmgp_rfmip_{lw|sw}.F90`. In addition, there is an option (`compare_flux =.true.`) to compare the output fluxes to benchmark line-by-line computations, alongside reference RRTMGP results which were produced in double precision."

- The Makefile must be changed to include `FCFLAGS += $(OPTFLAGS) $(DEBUGFLAGS)`, both in RTE+RRTMGP libraries and in rrtmgp_rfmip_{lw|sw}.

- A copy of the original `rrttmgp_rfmip_{lw|sw}.F90` file was kept in `rrttmgp_rfmip_{lw|sw}-v1.F90`.

## rrtmgp_rfmip_lw

The `rrtmgp_rfmip_lw.F90` file needs to be manually modified to set the variable: `use_rrtmgp_nn = .false.`.

## Build the RTE+RRTMGP libraries

In [1]:
%cd /home/x/git/radnn/ukk22test02/build

/home/x/git/radnn/ukk22test02/build


In [2]:
%env FC=gfortran
%env FCFLAGS=-ffree-line-length-none -march=native -O3
%env NCHOME=/usr
%env NFHOME=/usr
%env BLASLIB=openblas

env: FC=gfortran
env: FCFLAGS=-ffree-line-length-none -march=native -O3
env: NCHOME=/usr
env: NFHOME=/usr
env: BLASLIB=openblas


In [3]:
! make clean

rm -f *.optrpt *.mod *.o librrtmgp.a librte.a libneural.a


In [4]:
! make DEBUGFLAGS=-pg

gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/mo_rte_kind.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/mo_rte_rrtmgp_config.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/mo_rte_util_array.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/kernels/mo_optical_props_kernels.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/mo_optical_props.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/mo_source_functions.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/kernels/mo_fluxes_broadband_kernels.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../rte/mo_fluxes.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../neural/mod_activation.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../neural/mod_random.F90
gfortran -ffree-line-length-none -march=native -O3  -pg  -c ../neural/mod_layer.F90
gfortra

## Build the examples

In [5]:
%cd /home/x/git/radnn/ukk22test02/examples/rfmip-clear-sky

/home/x/git/radnn/ukk22test02/examples/rfmip-clear-sky


In [6]:
! make clean

VAR="../../"
rm rrtmgp_rfmip_sw rrtmgp_rfmip_lw *.o *.mod *.optrpt
rm: cannot remove '*.optrpt': No such file or directory
make: [Makefile:162: clean] Error 1 (ignored)


In [7]:
! make DEBUGFLAGS=-pg

VAR="../../"
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c ../mo_simple_netcdf.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c easy_netcdf.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c mo_rfmip_io.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c ../mo_load_coefficients.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -I../..//build -I/usr/include -c rrtmgp_rfmip_lw.F90 -fopenmp
gfortran -ffree-line-length-none -march=native -O3 -DNC_NETCDF4  -pg -o rrtmgp_rfmip_lw rrtmgp_rfmip_lw.o mo_simple_netcdf.o easy_netcdf.o mo_rfmip_io.o mo_load_coefficients.o ../..//build/librte.a ../..//build/librrtmgp.a ../..//build/libneural.a -L../..//build -L/usr/lib -L/usr/lib -lrrtmgp -lrte -lneural -lnetcdff -lnetcdf 

## Run

### LW

In [8]:
! time ./rrtmgp_rfmip_lw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc \
    1 \
    1

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [physics_index (1,2)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 Doing          225 blocks of size            8
 setting no2 to zero
 in total:           19  input gases
  shape play          60           8         225
 play sfc   103243.766     tlay sfc   311.876038    
 === using rrtmgp (without nn), line 340 ===
 starting clear-sky longwave computations, using lookup-table as RRTMGP kernel
 Elapsed time on everything   0.651000023    
 -----------------------------------------------------------------------------------------
 mean of flux_down is:   103.236938    
 mean of flux_up is:   301.596222    
 --------------------------------------------------------------------------------------------

In [9]:
! time ./rrtmgp_rfmip_lw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc \
    1 \
    1

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [physics_index (1,2)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 Doing          225 blocks of size            8
 setting no2 to zero
 in total:           19  input gases
  shape play          60           8         225
 play sfc   103243.766     tlay sfc   311.876038    
 === using rrtmgp (without nn), line 340 ===
 starting clear-sky longwave computations, using lookup-table as RRTMGP kernel
 Elapsed time on everything   0.612999976    
 -----------------------------------------------------------------------------------------
 mean of flux_down is:   103.236938    
 mean of flux_up is:   301.596222    
 --------------------------------------------------------------------------------------------

In [10]:
! time ./rrtmgp_rfmip_lw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc \
    1 \
    1

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [physics_index (1,2)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 Doing          225 blocks of size            8
 setting no2 to zero
 in total:           19  input gases
  shape play          60           8         225
 play sfc   103243.766     tlay sfc   311.876038    
 === using rrtmgp (without nn), line 340 ===
 starting clear-sky longwave computations, using lookup-table as RRTMGP kernel
 Elapsed time on everything   0.615000010    
 -----------------------------------------------------------------------------------------
 mean of flux_down is:   103.236938    
 mean of flux_up is:   301.596222    
 --------------------------------------------------------------------------------------------

In [11]:
real = [1.915, 1.858, 1.803]
sum(real)/len(real)

1.858666666666667

### SW

In [12]:
! time ./rrtmgp_rfmip_sw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-sw-g224-2018-12-04.nc \
    1 \
    1

 loading shortwave absorption model from ../../neural/data/BEST_tau-sw-abs-7-16-16-mae_2.txt                             
 loading rayleigh model from ../../neural/data/BEST_tau-sw-ray-7-16-16_2.txt                                 
 Usage: rrtmgp_rfmip_sw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 ninputs:           7
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o carbon_dioxide o3 nitrous_oxide carbon_monoxide methane oxygen nitrogen carbon_tetrachloride cfc11 cfc12 hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky sh

In [13]:
! time ./rrtmgp_rfmip_sw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-sw-g224-2018-12-04.nc \
    1 \
    1

 loading shortwave absorption model from ../../neural/data/BEST_tau-sw-abs-7-16-16-mae_2.txt                             
 loading rayleigh model from ../../neural/data/BEST_tau-sw-ray-7-16-16_2.txt                                 
 Usage: rrtmgp_rfmip_sw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 ninputs:           7
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o carbon_dioxide o3 nitrous_oxide carbon_monoxide methane oxygen nitrogen carbon_tetrachloride cfc11 cfc12 hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky sh

In [14]:
! time ./rrtmgp_rfmip_sw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-sw-g224-2018-12-04.nc \
    1 \
    1

 loading shortwave absorption model from ../../neural/data/BEST_tau-sw-abs-7-16-16-mae_2.txt                             
 loading rayleigh model from ../../neural/data/BEST_tau-sw-ray-7-16-16_2.txt                                 
 Usage: rrtmgp_rfmip_sw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [optional gas optics input_output file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 ncol:         100
 nexp:          18
 nlay:          60
 ninputs:           7
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o carbon_dioxide o3 nitrous_oxide carbon_monoxide methane oxygen nitrogen carbon_tetrachloride cfc11 cfc12 hcfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky sh

## Gprof

### LW

In [15]:
! gprof ./rrtmgp_rfmip_lw

Flat profile:

Each sample counts as 0.01 seconds.
 no time accumulated

  %   cumulative   self              self     total           
 time   seconds   seconds    calls  Ts/call  Ts/call  name    
  0.00      0.00     0.00    83233     0.00     0.00  lw_solver_noscat_GaussQuad
  0.00      0.00     0.00     3906     0.00     0.00  __mo_gas_concentrations_MOD_set_vmr_2d
  0.00      0.00     0.00     2702     0.00     0.00  __mo_rte_lw_MOD_rte_lw
  0.00      0.00     0.00     1800     0.00     0.00  sw_layer_props_sources_2str
  0.00      0.00     0.00     1719     0.00     0.00  zero_array_3D
  0.00      0.00     0.00     1358     0.00     0.00  __mo_optical_props_MOD___copy_mo_optical_props_Ty_optical_props_2str
  0.00      0.00     0.00     1125     0.00     0.00  __mo_gas_concentrations_MOD___final_mo_gas_concentrations_Conc_field
  0.00      0.00     0.00      900     0.00     0.00  __mod_random_MOD_randn2d
  0.00      0.00     0.00      594     0.00     0.00  __mo_gas_concentratio

### SW

In [16]:
! gprof ./rrtmgp_rfmip_sw

Flat profile:

Each sample counts as 0.01 seconds.
 no time accumulated

  %   cumulative   self              self     total           
 time   seconds   seconds    calls  Ts/call  Ts/call  name    
  0.00      0.00     0.00    79987     0.00     0.00  __mo_rrtmgp_util_string_MOD_lower_case
  0.00      0.00     0.00     5625     0.00     0.00  __mo_gas_concentrations_MOD_find_gas
  0.00      0.00     0.00     3681     0.00     0.00  __mo_gas_concentrations_MOD_set_vmr_scalar
  0.00      0.00     0.00     1800     0.00     0.00  adding
  0.00      0.00     0.00     1719     0.00     0.00  __mo_rte_util_array_MOD_any_vals_outside_2d
  0.00      0.00     0.00     1575     0.00     0.00  __mo_rte_util_array_MOD_extents_are_2d
  0.00      0.00     0.00     1125     0.00     0.00  __mo_gas_concentrations_MOD_get_conc_dims_and_igas
  0.00      0.00     0.00      908     0.00     0.00  __mo_optical_props_MOD_is_initialized_base
  0.00      0.00     0.00      900     0.00     0.00  __mod_activa